In [1]:
# ei_selective.ipynb
# This notebook is used to create a balanced network of excitatory and inhibitory units that become selective
# for particular input patterns.

# By Sergio Verduzco Flores        August 2017

from sirasi import *
from matplotlib import pyplot as plt
import numpy as np
import pylab
import time

The cells below:
* Create two NxN layers, one excitatory, and one inhibitory.
* Connect the layers with center-excitation surround-inhibition connectivity. E-E connections use
  Hebbian learning with substractive normalization, I-E connections use homeostatic inhibition,
  similar to Moldakarimov06 (in ei_net.ipynb).
* Create an input layer where the activity of each unit is a function of the current input pattern.
* Connects the input layer to E,I layers using a connection pattern based on a Gaussian kernel, and
  using BCM synapses.
* Runs a simulation where input patterns are randomly selected and presented sequentially, similarly
  to the way this was done in the test3,4,5 notebooks.
  
The experiment consists of finding whether selectivity to patterns will arise in the units.
- - -
TODO:
* Set the right connectivity patterns, and **set the delays according to distance**.

In [2]:
# create some utility functions
def make_fe(th, eps): # returns a function as in Eq. 1 of the paper
    return lambda x : np.sqrt( eps * np.log( 1. + np.exp( (x - th)/eps ) ) )
def make_fi(th, eps): # returns a function as in Eq. 2 of the paper
    return lambda x: eps * np.log( 1. + np.exp( (x - th)/eps ) ) 
def make_pulse(t_init, t_end): # returns f(t) = 1 if t_init < t < t_end, 0 otherwise
    return lambda t : 1. if (t_init < t and t < t_end) else 0.
def r(i,j,sigma): # A Gaussian function with s.d. sigma, applied to i-j
    return (1./(sigma*np.sqrt(2*np.pi))) * np.exp( -0.5*( ((i-j)/sigma)**2 ) )

In [3]:
""" Create and connect the E,I layers.
    Random connections with fixed indegree as a starter. 
"""

######## CREATE THE NETWORK
md = 0.5 # minimum delay of the connections
net_params = {'min_delay' : md, 'min_buff_size' : 3, 'rtol':1e-4, 'atol':1e-4 } 
net = network(net_params)

######## CREATE THE E,I UNITS
N = 5 # Each layer will have N*N units
exc_params = {'tau' : 2., 'function' : make_fe(0.1, 0.2), 'type' : unit_types.custom_fi, 
              'init_val' : 0.1, 'tau_fast' : 0.2, 'tau_slow' : 20. }
inh_params = {'tau' : 3., 'function' : make_fi(0.5, 0.2), 'type' : unit_types.custom_fi, 
              'init_val' : 0.1, 'tau_fast' : 0.2, 'tau_slow' : 20. }

exc = net.create(N*N, exc_params)
inh = net.create(N*N, inh_params)

######## SET E-I CONNECTIONS
# connection specifications
EI_conn = {'rule' : 'fixed_indegree', 'indegree' : 5, 'delay' : md}
# synapse specifications
EI_syn = {'type' : synapse_types.static, 'init_w' : .2}
# Static connections and fixed indegree suggest that the inhibitory units will not go without excitation.
# connect
net.connect(exc, inh, EI_conn, EI_syn)

######## SET E-E CONNECTIONS
# connection specifications
EE_conn = {'rule' : 'fixed_indegree', 'indegree' : 10, 'delay' : md}
# synapse specifications
EE_syn = {'type' : synapse_types.sq_hebbsnorm, 'init_w' : {'distribution':'uniform', 'low':0., 'high':.5},
          'lrate' : 1./100., 'omega' : 4.} 
# The large g_ee value in Moldakaraimov06 can probably be compensated with omega and the indegree.
# connect
net.connect(exc, exc, EE_conn, EE_syn)

######## SET I-E CONNECTIONS
# connection specifications
IE_conn = {'rule' : 'fixed_outdegree', 'outdegree' : 10, 'delay' : md}
# synapse specifications
IE_syn = {'type' : synapse_types.homeo_inh, 'init_w' : {'distribution':'uniform', 'low':-.6, 'high':0.},
           'lrate' : 1./50., 'des_act' : 0.3}
# connect
net.connect(inh, exc, IE_conn, IE_syn)

In [4]:
""" Create and connect the input layer. Create some input paterns 
"""

######### CREATE THE INPUT UNITS
inp_params = {'type' : unit_types.source, 'init_val' : 0., 'tau_fast' : 0.2, 'function' : lambda x: None}
inp = net.create(N*N, inp_params)

######### CONNECT THE INPUT UNITS
# connection specifications
PE_conn = {'rule' : 'one_to_one', 'delay' : md}
PI_conn = {'rule' : 'one_to_one', 'delay' : md}
# synapse specifications
PE_syn = {'init_w' : {'distribution':'uniform', 'low':1., 'high':4.}, 'omega' : N*N, 
          'lrate' : 1./50., 'type' : synapse_types.bcm }
PI_syn = {'init_w' : {'distribution':'uniform', 'low':1., 'high':4.}, 'omega' : N*N, 
          'lrate' : 1./50., 'type' : synapse_types.bcm }
# connect
net.connect(inp, exc, PE_conn, PI_syn)

######### CREATE THE INPUT PATTERNS
# For the input patterns I use random vectors where each element is 0 or 1. 
# All patterns will contain the same number of 1 values.
n_pat = 5 # number of input patterns
n_ones = int(round(N*N)/3) # number of ones in the input patterns
basic_pat = np.array( [1.]*n_ones + [0.]*(N*N - n_ones) )
patterns = [basic_pat.copy() for i in range(n_pat)]
for pat in patterns:
    np.random.shuffle(pat)


"""

# synapse specifications
# I don't have the theta and the omega parameters, I assumed theta = 1. 
# For omega I used 1 . If all weights are equal and add to c, 20w=c => w=c/20 => omega=20w^2=(c^2)/20 .
# lrate can be interpreted as the reciprocal of the synaptic time constants.
g_ee=5.; g_ie=1.; g_ei=3.5 # gains for each input type


ext2exc_syn = {'type' : synapse_types.static, 'init_w' : 5., 'gain' : 1.}
ext2inh_syn = {'type' : synapse_types.static, 'init_w' : 5., 'gain' : 1.}

# neuron-to-neuron connection
sig = 3. # standard deviation of connection multiplier
base_e = exc[0] # index of first excitatory unit
base_i = inh[0]
for e,i in zip(exc,inh):  # connect with non-periodic boundary conditions
    net2.connect([i], [e], inh2exc_conn, inh2exc_syn) # inhibitory units only connect locally
    for target_e, target_i in zip(exc,inh):
        exc2exc_syn['gain'] = g_ee * r(e, target_e, 3.)
        net2.connect([e], [target_e], exc2exc_conn, exc2exc_syn)
        exc2inh_syn['gain'] = g_ie * r(e-base_e, target_i-base_i, 3.)
        net2.connect([e], [target_i], exc2inh_conn, exc2inh_syn)
        
# external input connections
net2.connect(ext, [exc[9]], ext2exc_conn, ext2exc_syn)
net2.connect(ext, [inh[9]], ext2inh_conn, ext2inh_syn)

# running the simulation
n_iters = 64
act_fig = plt.figure(figsize=(10,8))
dubyas = []  # to show the evolution of excitatory connections
for iter in range(1,n_iters+1):
    start_time = time.time()
    times, units, empty = net2.run(50.)
    if iter == 1: ole_t = times
    print('Execution time: at iteration %d is %s seconds' % (iter, (time.time() - start_time)) )
    sim_time = net2.sim_time
    net2.units[ext[0]].set_function(make_pulse(sim_time+10,sim_time+10.+width))
    if iter%8 == 0:
        pylab.plot(ole_t, units[exc[9]], 'b', ole_t, units[inh[9]], 'r', ole_t, units[ext[0]], 'g', figure=act_fig)
        dubyas.append([syn.w for syn in net2.syns[exc[9]] if syn.type == synapse_types.sq_hebbsnorm ])

'''
gain_fig = plt.figure(figsize=(5,3))
inps = np.linspace(-5.,10.,100)
exc_fr = np.array([net2.units[exc[0]].f(I) for I in inps])
inh_fr = np.array([net2.units[inh[0]].f(I) for I in inps])
pylab.plot(inps, exc_fr, 'k', inps, inh_fr, 'k--', figure=gain_fig)
'''
from mpl_toolkits.axes_grid1 import ImageGrid
dub_mat = np.matrix(dubyas)
dub_fig = plt.figure(figsize=(8,6))
dub_grid = ImageGrid(dub_fig, 111, nrows_ncols=(1,1))
dub_grid[0].imshow(dub_mat)
plt.show()
"""


"\n\n# synapse specifications\n# I don't have the theta and the omega parameters, I assumed theta = 1. \n# For omega I used 1 . If all weights are equal and add to c, 20w=c => w=c/20 => omega=20w^2=(c^2)/20 .\n# lrate can be interpreted as the reciprocal of the synaptic time constants.\ng_ee=5.; g_ie=1.; g_ei=3.5 # gains for each input type\n\n\next2exc_syn = {'type' : synapse_types.static, 'init_w' : 5., 'gain' : 1.}\next2inh_syn = {'type' : synapse_types.static, 'init_w' : 5., 'gain' : 1.}\n\n# neuron-to-neuron connection\nsig = 3. # standard deviation of connection multiplier\nbase_e = exc[0] # index of first excitatory unit\nbase_i = inh[0]\nfor e,i in zip(exc,inh):  # connect with non-periodic boundary conditions\n    net2.connect([i], [e], inh2exc_conn, inh2exc_syn) # inhibitory units only connect locally\n    for target_e, target_i in zip(exc,inh):\n        exc2exc_syn['gain'] = g_ee * r(e, target_e, 3.)\n        net2.connect([e], [target_e], exc2exc_conn, exc2exc_syn)\n        

In [5]:
patterns

[array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
         1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.]),
 array([ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,
         1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.]),
 array([ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
         0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.]),
 array([ 1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,
         0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.])]